In [40]:
import pandas as pd

#df = pd.read_csv('../data/donnees_nettoyees_avec_coord.csv', index_col=0)

df = pd.read_pickle('../data/dataframe_clean_last_version.pkl')
print(len(df), " individus")


print(df['Departement'].nunique()," départements différents")
df.head(15)



1865937  individus
95  départements différents


,No_disposition,Nature_mutation,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,latitude,longitude,Departement
0,1,Vente,220000.0,44260,PRINQUIAU,ZI,383,0,1,127.0,6.0,S,746.0,19,7,2018,47.373515,-2.018452,44
1,1,Vente,80000.0,13400,AUBAGNE,AO,360,0,1,155.0,7.0,S,494.0,26,9,2018,43.293484,5.563313,13
2,1,Vente,160000.0,2200,SOISSONS,AE,99,0,1,90.0,4.0,S,194.0,9,1,2018,49.379174,3.324718,02
3,1,Vente,280000.0,95300,PONTOISE,AE,458,0,1,75.0,4.0,S,509.0,5,3,2018,49.051374,2.094879,95
4,1,Vente,217650.0,31620,BOULOC,E,1530,0,1,85.0,4.0,S,273.0,31,10,2018,43.795800,1.419189,31
5,1,Vente,127500.0,22450,LA ROCHE DERRIEN,AC,327,0,1,67.0,4.0,S,267.0,10,12,2018,48.745637,-3.257628,22
6,1,Vente,275000.0,93700,DRANCY,AJ,256,0,1,75.0,3.0,S,339.0,18,5,2018,48.923425,2.444927,93
7,1,Vente,73000.0,59400,MOEUVRES,A,997,0,1,115.0,5.0,S,1633.0,31,5,2018,50.160216,3.067819,59
8,1,Vente,136000.0,49112,VERRIERES EN ANJOU,AB,172,0,1,122.0,5.0,S,265.0,22,1,2018,47.512332,-0.475838,49
9,1,Vente,110000.0,82300,MONTEILS,B,689,0,1,72.0,5.0,J,1548.0,12,4,2018,44.171787,1.565377,82


In [37]:
import folium
from folium.plugins import HeatMap
from branca.colormap import linear

# Créez une liste de listes contenant les coordonnées [latitude, longitude] et la valeur "Valeur fonciere"
heat_data = [[row['latitude'], row['longitude'], row['Valeur_fonciere']]
             for index, row in df.iterrows()]

# Créez la carte
my_map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Créez une colormap linéaire basée sur la colonne "Valeur fonciere"
colormap = linear.YlOrRd_09.scale(
    min(df['Valeur_fonciere']), max(df['Valeur_fonciere']))

# Ajoutez la carte de chaleur à partir des données
HeatMap(heat_data,
        min_opacity=0.2,
        max_val=max(df['Valeur_fonciere']),
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(my_map)

# Ajoutez une légende basée sur la colormap
colormap.caption = "Valeur fonciere"
colormap.add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map.html')


C:\Users\nagrimault\AppData\Local\Temp\ipykernel_11880\449697079.py:17: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data,


In [41]:
import folium
from folium.plugins import HeatMap
from folium import Choropleth, GeoJson
import pandas as pd

pd.set_option('display.max_rows', None)  # Affichez toutes les lignes
# Calcul de la valeur foncière moyenne par département
df_avg_value_by_dept = df.groupby(
    'Departement')['Valeur_fonciere'].mean().reset_index()

print(df_avg_value_by_dept)
print(len(df_avg_value_by_dept), " départements différents")
# Coordonnées du centre de la France (ou un autre emplacement de votre choix)
latitude, longitude = 46.603354, 1.888334

# Créez la carte
my_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# Ajoutez la carte choroplèthe en fonction de la valeur foncière moyenne par département
Choropleth(
    # Chemin vers le fichier GeoJSON des départements de la France
    geo_data='../data/departements.geojson',
    name='choropleth',
    data=df_avg_value_by_dept,
    columns=['Departement', 'Valeur_fonciere'],
    # Clé pour faire correspondre les données GeoJSON avec le DataFrame
    key_on='feature.properties.code',
    # Palette de couleurs (vous pouvez choisir d'autres palettes)
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Valeur fonciere moyenne par département'
).add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map_departements.html')


   Departement  Valeur_fonciere
0           01    232173.000631
1           02    148041.112690
2           03    146906.136835
3           04    213793.683335
4           05    235042.920033
5           06    323630.512454
6           07    197536.617678
7           08    146166.853935
8           09    157269.692093
9           10    164573.249340
10          11    180017.558909
11          12    166715.962161
12          13    301692.871097
13          14    216023.859978
14          15    150337.682209
15          16    158788.417051
16          17    219852.135772
17          18    144575.235190
18          19    155614.558086
19          20    279386.863205
20          21    207108.207835
21          22    179041.373193
22          23    129020.131111
23          24    176251.721839
24          25    207211.290161
25          26    224327.316767
26          27    183585.733327
27          28    185910.842679
28          29    186058.537606
29          2A    292293.100328
30      

In [44]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_geocoder")

# Remplacez le code postal correct de la ville voulue et vérifier les coordonnées
code_postal = "2200"
adresse = f"SOISSONS, {code_postal}, France"

# Utilisez Nominatim pour obtenir les coordonnées géographiques
location = geolocator.geocode(adresse)

# Affichez les coordonnées géographiques (latitude et longitude) de "AVERMES"
if location:
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Coordonnées non trouvées pour l'adresse spécifiée.")


Latitude: 49.3837621, Longitude: 3.327613
